---
# Binned, single subject example 
*See [mkpy docs](https://kutaslab.github.io/mkpy/Quick%20Start.html) for explanations of all the commands and files. Click on the command to find it in the glossary and learn more. The glossary links are incuded for each command below* <br>
## Conda environment
*always activate a conda environment before starting. This notebook uses [mkconda](https://anaconda.org/kutaslab/mkconda) 0.0.11*

## Python Libraries

In [1]:
from pathlib import Path
from mkpy import mkh5
import pandas as pd

## Data Paths
Set paths and variable names for files later

In [2]:
MKDIG_DIR = Path("../../mkdig")
MKPY_DIR = Path("../../mkpy")

## File names
Set file names so they can be easily changed and don't have to retype every time

In [3]:
# h5 continuous file 
h5_f = str(MKPY_DIR / ("stm02.h5"))

# h5 epoch file 
epochtable = str(MKPY_DIR / ("stm02.epochs.h5"))

# subject ID
sub = 'stm02'

# kutas lab data files (from DIG + yhdr that you created)
eeg = MKDIG_DIR / (sub + ".crw")
log = MKDIG_DIR / (sub +".x.log")
yhdr = MKPY_DIR / (sub + ".yhdr")

## Build the continuous h5 file

1) [mkh5.mkh5](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5): Import the h5 file into a dataframe and reset it (generally you will want to reset the file to start fresh).

In [4]:
myh5 = mkh5.mkh5(h5_f)
myh5.reset_all()

2) [create_mkdata](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5.create_mkdata): load in subject and cals to the h5 data structure<br><br>
*The first argument is the data branch you want to add the data to. This can be subject name, or for more complicated files could be a path.*<br><br>
&emsp;2a) [append_mkdata](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5.append_mkdata): if your file is split into two for some reason (i.e., separate cals, DIG crash) this is where you would add this command as well

In [5]:
myh5.create_mkdata(sub, eeg, log, yhdr)

3) [calibrate_mkdata](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5.calibrate_mkdata): calibrate data; the first line sets the values similar to normerp<br><br>
*If you don't know what values to use, you can use [plotcals](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5.plotcals) to select them before running the calibrate command.* 

In [6]:
pts, pulse, lo, hi, ccode = 5, 10, -40, 40, 0
myh5.calibrate_mkdata(sub, # specific data group
                    n_points = pts,   # pts to average
                    cal_size = pulse, # uV
                    lo_cursor = lo,   # lo_cursor ms
                    hi_cursor = hi,   # hi_cursor ms
                    cal_ccode= ccode) # condition code

/home/astoermann/.conda/envs/test_jupyter_books_11/lib/python3.6/site-packages/mkpy/mkh5.py:3706: UserWarning: negative event code(s) found for cal condition code 0 -16384
  warnings.warn(msg)


Found cals in /stm02/dblock_5
Calibrating block /stm02/dblock_0 of 6: (148224,)  


Calibrating block /stm02/dblock_1 of 6: (146944,)  


Calibrating block /stm02/dblock_2 of 6: (148224,)  


Calibrating block /stm02/dblock_3 of 6: (271104,)  


Calibrating block /stm02/dblock_4 of 6: (122624,)  


Calibrating block /stm02/dblock_5 of 6: (19200,)  


```{note}
The warning above is telling you there is a negative event code in the cals (the pause) that wasn't expected. This warning is fine to get. Sometimes pauses happen in the cals i.e, pausing before the cal pulses are completely done.
```

## Find event codes in the data
[get_event_table](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5.get_event_table): Label the events in the data, but don't pull out epochs yet. Capture this info in a data frame.
* For this simple example we will use the .ytbl format for the code map (see {doc}`codemaps` ytbl example)

In [7]:
event_table = myh5.get_event_table(MKPY_DIR / ("stmath.ytbl"))
## Look at the event table you just made
event_table.head(312)

searching codes in: stm02/dblock_0
searching codes in: stm02/dblock_1
searching codes in: stm02/dblock_2
searching codes in: stm02/dblock_3


searching codes in: stm02/dblock_4
searching codes in: stm02/dblock_5


,data_group,dblock_path,dblock_tick_idx,dblock_ticks,crw_ticks,raw_evcodes,log_evcodes,log_ccodes,log_flags,epoch_match_tick_delta,...,match_str,anchor_code,match_code,anchor_tick,match_tick,anchor_tick_delta,is_anchor,condition,regexp,STthreat
0,stm02,stm02/dblock_0,66,13565,13565,11025,11025,1,0,0,...,11025,11025,11025,13565,13565,0,True,CorrectNoST,(#11[0-9]{3}),NoST
1,stm02,stm02/dblock_0,108,21958,21958,11003,11003,1,32,0,...,11003,11003,11003,21958,21958,0,True,CorrectNoST,(#11[0-9]{3}),NoST
2,stm02,stm02/dblock_0,192,38634,38634,11078,11078,1,0,0,...,11078,11078,11078,38634,38634,0,True,CorrectNoST,(#11[0-9]{3}),NoST
3,stm02,stm02/dblock_0,234,47106,47106,11088,11088,1,0,0,...,11088,11088,11088,47106,47106,0,True,CorrectNoST,(#11[0-9]{3}),NoST
4,stm02,stm02/dblock_0,248,50191,50191,11039,11039,1,0,0,...,11039,11039,11039,50191,50191,0,True,CorrectNoST,(#11[0-9]{3}),NoST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,stm02,stm02/dblock_4,568,95246,809742,16061,16061,1,0,0,...,16061,16061,16061,95246,95246,0,True,RelatedST,(#16[0-9]{3}),ST
307,stm02,stm02/dblock_4,624,104618,819114,16143,16143,1,0,0,...,16143,16143,16143,104618,104618,0,True,RelatedST,(#16[0-9]{3}),ST
308,stm02,stm02/dblock_4,638,106928,821424,16141,16141,1,0,0,...,16141,16141,16141,106928,106928,0,True,RelatedST,(#16[0-9]{3}),ST
309,stm02,stm02/dblock_4,680,114040,828536,16040,16040,1,0,0,...,16040,16040,16040,114040,114040,0,True,RelatedST,(#16[0-9]{3}),ST


## Set epochs
[set_epochs](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5.set_epochs): Pull out epochs in the data. Does not save them yet.
* First argument is the name of the epochs table
* Second argument is the event table you made (not the code map)
* Third and fourth argument is the window you want the epoct to have around 0

In [8]:
myh5.set_epochs('stmath', event_table, -100, 900) 

Sanitizing event table data types for mkh5 epochs table ...


## Export epochs
[export_epochs](https://kutaslab.github.io/mkpy/source/mkpy.mkh5.html#mkpy.mkh5.mkh5.export_epochs): save the epoched data to use later. Can be in h5, pdh5 (pandas), or feather format.
* First argument is the epochs table
* Second argument is the file name to save to (set above in the file names section)
* Third argument is what you want to save as

In [9]:
myh5.export_epochs('stmath', epochtable, file_format='h5')